## Data Ingestion

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/waste-detection-using-yoloV5'

In [5]:
# Config Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    raw_dataset_dir: Path
    dataset_dir: Path

In [6]:
from wasteDetection.constants import *
from wasteDetection.utils import create_directories, read_yaml

In [12]:
# Configuration Manager
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = config.source_URL,
            raw_dataset_dir = Path(config.raw_dataset_dir),
            dataset_dir = Path(config.dataset_dir)
        )

        return data_ingestion_config

In [23]:
import gdown
from wasteDetection.logger import logger
import zipfile
import shutil

In [30]:
# Components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def _download_dataset(self) -> None:
        '''
        Fetch the dataset from the URL
        '''

        try:
            if (not os.path.exists(self.config.dataset_dir)) or (not os.listdir(self.config.dataset_dir)):
                create_directories([self.config.raw_dataset_dir])
                
                dataset_url = self.config.source_url
                output_path = os.path.join(self.config.raw_dataset_dir, 'data.zip')
                file_id = dataset_url.split('/')[-2]
                prefix = 'https://drive.google.com/uc?/export=download&id='
                gdown.download(prefix+file_id, output_path, quiet=False)

                logger.info("Dataset downloaded successful")


        except Exception as e:
            raise e
        

    def _preprocess_dataset(self):
        '''
        Preprocess the raw dataset
        '''
        if os.path.exists(self.config.raw_dataset_dir):
            create_directories([self.config.dataset_dir])
            try:
                for root, dirs, files in os.walk(self.config.raw_dataset_dir):
                    for file in files:
                        
                        # Check if the file is an zip file
                        if file.lower().endswith(('.zip')):
                            file_path = os.path.join(root, file)
                            with zipfile.ZipFile(file_path) as zip_ref:
                                zip_ref.extractall(self.config.dataset_dir)

                shutil.rmtree(self.config.raw_dataset_dir)
            
            except Exception as e:
                raise e
            
    
    def initiate_data_ingestion(self) -> None:
        self._download_dataset()
        self._preprocess_dataset()
    

In [32]:
# Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()

except Exception as e:
    raise(e)

2023-11-22 10:31:15,018 : wasteDetection.logger - INFO - YAML file: config/config.yaml loaded successfully
2023-11-22 10:31:15,020 : wasteDetection.logger - INFO - YAML file: params.yaml loaded successfully
2023-11-22 10:31:15,022 : wasteDetection.logger - INFO - created directory at: artifacts
2023-11-22 10:31:15,024 : wasteDetection.logger - INFO - created directory at: artifacts/data_ingestion
2023-11-22 10:31:15,026 : wasteDetection.logger - INFO - created directory at: artifacts/data_ingestion/raw_dataset
Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1YBtZfdYtJ341DJ51JJFA8iOrHC4A9b22
From (redirected): https://drive.google.com/uc?/export=download&id=1YBtZfdYtJ341DJ51JJFA8iOrHC4A9b22&confirm=t&uuid=1e7efe45-a044-4bfd-b9fb-2604d8273cd2
To: /home/utpal108/dev/Python/PW_Skills_Projects/waste-detection-using-yoloV5/artifacts/data_ingestion/raw_dataset/data.zip
100%|██████████| 168M/168M [01:06<00:00, 2.53MB/s]
2023-11-22 10:32:23,685 : wasteDetection.l